In [2]:
%matplotlib inline
import seaborn as sns; 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import gridspec
import scipy
style.use('ggplot')
import datetime
from io import StringIO
import re
import os
import numpy as np
from datetime import datetime
import apt_pkg
apt_pkg.init_system()
import json as js
import codecs
import psycopg2

# Load installed source packages

In [42]:
sources = pd.read_csv('../../data/for_analysis/installed_packages.csv',
                      usecols=['source','source_version','archive','last_updated'], dtype=str)

In [43]:
sources.drop_duplicates(inplace=True)
sources.shape

(1223023, 4)

# Load Debian Packages

In [44]:
debian_packages = pd.read_csv('../../data/prepared_data/debian_packages.csv', dtype=str)

In [45]:
debian_packages = debian_packages[['source','source_version','date','archive']].drop_duplicates().copy()

In [46]:
debian_packages = (debian_packages
                   .sort_values('date')
                  .groupby(['source','source_version','archive'])
                   .first()
                   .reset_index()
                  )
debian_packages.shape

(270292, 4)

# Identify the ideal source package at the image date

In [47]:
sources = (sources
            .merge(debian_packages
                   .rename(columns={'source_version':'ideal_source'}),
                   on = ['source','archive'],
                   how = 'left'
                  )
           )
sources.shape

(9857884, 6)

In [48]:
# Remove package version that were created after the containers inspection
sources = sources.query('last_updated>=date').copy()
sources.shape

(9077598, 6)

In [49]:
sources = (sources
            .sort_values('date', ascending=False)
            .groupby(['source','source_version','archive','last_updated'])
            .first()
            .reset_index()
           )
sources.shape

(1222801, 6)

In [50]:
def compare(version,ideal):
    if apt_pkg.version_compare(str(version), str(ideal)) >=0:
        return version
    else:
        return ideal
    # 1 if version > compare_to

In [51]:
sources['ideal_source'] = sources.apply(lambda d: compare(d['source_version'], d['ideal_source']), axis=1)

In [52]:
sources.drop('date', axis=1, inplace=True)
sources.drop_duplicates(inplace=True)

In [53]:
len(sources.drop_duplicates())

1222801

In [54]:
sources.to_csv('../../../docker_emse/csv/for_analysis/sources_ideal.csv', index=False)

# EXTRACT VULS

In [3]:
sources = pd.read_csv('../../../docker_emse/csv/for_analysis/sources_ideal.csv', dtype=object)

In [6]:
def parse_json_vuls():
    vulnerabilities=js.load(codecs.open('../../data/prepared_data/vuls_extracted_20190830.json', 'r', 'utf-8'))
    return vulnerabilities

In [7]:
vulnerabilities = parse_json_vuls() 

fcsv = open('../../data/for_analysis/vulnerabilities.csv', 'w')
fcsv.write('source,source_version,urgency,status,fixed_in,archive,debianbug,cve\n')
for row in sources[['source','source_version','archive']].drop_duplicates().iterrows():
    source = row[1]['source']
    source_version = row[1]['source_version']
    release = row[1]['archive']
    try:
        vuls=vulnerabilities[source] ###### check if the source has any vulnerabilities
    except:
        continue
    for cve in vuls:  ###### for each vulnerability
        if not cve.startswith('CVE'):
            continue
        v=vulnerabilities[source][cve]
        try:
            status = v['releases'][release]['status']  # check only the release of source
            urgency = v['releases'][release]['urgency']  # check only the release of source

            try:
                debianbug = str(v['debianbug'])
            except:
                debianbug = "undefined"
            if status == "open" or status == "undetermined":  # if the vulnerability is still OPEN
                fixed = "undefined"
                
            else: # if the vulnerability is RESOLVED and
                fixed = v['releases'][release]["fixed_version"]
                if apt_pkg.version_compare(source_version, fixed) >= 0:  # 
                    continue
            fcsv.write(','.join([source, source_version, urgency, status, fixed, release, debianbug, cve]) + '\n')

        except:
            pass
fcsv.close()

# Ectract vulnerabilities for the ideal

In [8]:
vulnerabilities = parse_json_vuls() 

fcsv = open('../../data/for_analysis/last_vulnerabilities.csv', 'w')
fcsv.write('source,source_version,urgency,status,fixed_in,archive,debianbug,cve\n')
for row in sources[['source','ideal_source','archive']].drop_duplicates().iterrows():
    source = row[1]['source']
    source_version = row[1]['ideal_source']
    release = row[1]['archive']
    try:
        vuls=vulnerabilities[source] ###### check if the source has any vulnerabilities
    except:
        continue
    for cve in vuls:  ###### for each vulnerability
        if not cve.startswith('CVE'):
            continue
        v=vulnerabilities[source][cve]
        try:
            status = v['releases'][release]['status']  # check only the release of source
            urgency = v['releases'][release]['urgency']  # check only the release of source

            try:
                debianbug = str(v['debianbug'])
            except:
                debianbug = "undefined"
            if status == "open" or status == "undetermined":  # if the vulnerability is still OPEN
                fixed = "undefined"
                
            else: # if the vulnerability is RESOLVED and
                fixed = v['releases'][release]["fixed_version"]
                if apt_pkg.version_compare(source_version, fixed) >= 0:  # 
                    continue
            fcsv.write(','.join([source, source_version, urgency, status, fixed, release, debianbug, cve]) + '\n')

        except:
            pass
fcsv.close()

# Merge between two datasets

In [9]:
vuls = pd.read_csv('../../data/for_analysis/vulnerabilities.csv', dtype=object)
vuls.head(2)

,source,source_version,urgency,status,fixed_in,archive,debianbug,cve
0,abiword,3.0.2-2+deb9u2,unimportant,open,undefined,stretch,884923,CVE-2017-17529
1,accountsservice,0.6.43-1,low,open,undefined,stretch,757912,CVE-2012-6655


In [10]:
last_vuls = pd.read_csv('../../data/for_analysis/last_vulnerabilities.csv', dtype=object)
last_vuls.head(2)

,source,source_version,urgency,status,fixed_in,archive,debianbug,cve
0,abiword,3.0.2-2+deb9u2,unimportant,open,undefined,stretch,884923,CVE-2017-17529
1,accountsservice,0.6.43-1,low,open,undefined,stretch,757912,CVE-2012-6655


In [11]:
vuls = pd.concat([vuls, last_vuls])
vuls.drop_duplicates(inplace=True)

In [14]:
vuls.to_csv('../../data/for_analysis/vulnerabilities.csv', index=False)
! rm ../../data/for_analysis/last_vulnerabilities.csv